# Recommendation System Notebook
- User based recommendation
- User based prediction & evaluation
- Item based recommendation
- Item based prediction & evaluation

Different Approaches to develop Recommendation System -

1. Demographich based Recommendation System

2. Content Based Recommendation System

3. Collaborative filtering Recommendation System

In [1]:
# import libraties
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Reading ratings file from GitHub. # MovieLens
ratings = pd.read_csv('https://raw.githubusercontent.com/antrikshsaxena/NLPCapstone/main/ratings_final.csv' , encoding='latin-1')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


## Dividing the dataset into train and test

In [3]:
# Test and Train split of the dataset.
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.30, random_state=31)

In [4]:
print(train.shape)
print(test.shape)

(210088, 4)
(90038, 4)


In [5]:
# Pivot the train ratings' dataset into matrix format in which columns are movies and the rows are user IDs.
df_pivot = train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)

df_pivot.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Creating dummy train & dummy test dataset
These dataset will be used for prediction 
- Dummy train will be used later for prediction of the movies which has not been rated by the user. To ignore the movies rated by the user, we will mark it as 0 during prediction. The movies not rated by user is marked as 1 for prediction in dummy train dataset. 

- Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train.

In [6]:
# Copy the train dataset into dummy_train
dummy_train = train.copy()

In [7]:
dummy_train.head()

,userId,movieId,rating,timestamp
285695,1986,5618,5.0,1468556487
216175,1525,1357,3.0,860937440
223915,1591,2571,5.0,1446621026
200924,1418,2060,3.0,1034922803
217719,1536,1704,4.0,1225328446


In [8]:
# The movies not rated by user is marked as 1 for prediction. 
dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x>=1 else 1)

In [9]:
# Convert the dummy train dataset into matrix format.
dummy_train = dummy_train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(1)

In [10]:
dummy_train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


**Cosine Similarity**

Cosine Similarity is a measurement that quantifies the similarity between two vectors [Which is Rating Vector in this case] 

**Adjusted Cosine**

Adjusted cosine similarity is a modified version of vector-based similarity where we incorporate the fact that different users have different ratings schemes. In other words, some users might rate items highly in general, and others might give items lower ratings as a preference. To handle this nature from rating given by user , we subtract average ratings for each user from each user's rating for different movies.



# User Similarity Matrix

## Using Cosine Similarity

In [11]:
df_pivot.index.nunique()

2071

In [12]:
from sklearn.metrics.pairwise import pairwise_distances

# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_pivot, metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[1.         0.02834514 0.04260006 ... 0.02010714 0.         0.02016493]
 [0.02834514 1.         0.12915063 ... 0.16690495 0.         0.11004906]
 [0.04260006 0.12915063 1.         ... 0.17826473 0.         0.04473138]
 ...
 [0.02010714 0.16690495 0.17826473 ... 1.         0.00286873 0.09695713]
 [0.         0.         0.         ... 0.00286873 1.         0.01475374]
 [0.02016493 0.11004906 0.04473138 ... 0.09695713 0.01475374 1.        ]]


In [13]:
user_correlation.shape

(2071, 2071)

## Using adjusted Cosine 

### Here, we are not removing the NaN values and calculating the mean only for the movies rated by the user

In [14]:
# Create a user-movie matrix.
df_pivot = train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
)

In [15]:
df_pivot.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalising the rating of the movie for each user around 0 mean

In [16]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [17]:
df_subtracted.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.169355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.368421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Finding cosine similarity

In [18]:
from sklearn.metrics.pairwise import pairwise_distances

In [19]:
# Creating the User Similarity Matrix using pairwise_distance function.
user_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation)

[[ 1.00000000e+00  1.89942155e-03  1.65714482e-02 ...  2.02835872e-02
   0.00000000e+00  4.45393263e-03]
 [ 1.89942155e-03  1.00000000e+00  2.43906478e-02 ...  3.46154488e-02
   0.00000000e+00 -6.29642259e-04]
 [ 1.65714482e-02  2.43906478e-02  1.00000000e+00 ...  1.12606845e-01
   0.00000000e+00  1.72034212e-02]
 ...
 [ 2.02835872e-02  3.46154488e-02  1.12606845e-01 ...  1.00000000e+00
   1.50194969e-02  4.05549027e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.50194969e-02
   1.00000000e+00 -3.89401922e-02]
 [ 4.45393263e-03 -6.29642259e-04  1.72034212e-02 ...  4.05549027e-03
  -3.89401922e-02  1.00000000e+00]]


In [20]:
user_correlation.shape

(2071, 2071)

## Prediction - User User

Doing the prediction for the users which are positively related with other users, and not the users which are negatively related as we are interested in the users which are more similar to the current users. So, ignoring the correlation for values less than 0. 

In [21]:
user_correlation[user_correlation<0]=0
user_correlation

array([[1.        , 0.00189942, 0.01657145, ..., 0.02028359, 0.        ,
        0.00445393],
       [0.00189942, 1.        , 0.02439065, ..., 0.03461545, 0.        ,
        0.        ],
       [0.01657145, 0.02439065, 1.        , ..., 0.11260685, 0.        ,
        0.01720342],
       ...,
       [0.02028359, 0.03461545, 0.11260685, ..., 1.        , 0.0150195 ,
        0.00405549],
       [0.        , 0.        , 0.        , ..., 0.0150195 , 1.        ,
        0.        ],
       [0.00445393, 0.        , 0.01720342, ..., 0.00405549, 0.        ,
        1.        ]])

Rating predicted by the user (for movies rated as well as not rated) is the weighted sum of correlation with the movie rating (as present in the rating dataset). 

In [22]:
user_predicted_ratings = np.dot(user_correlation, df_pivot.fillna(0))
user_predicted_ratings

array([[9.55212958e+00, 3.29449494e+00, 1.47095436e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.45759814e-02],
       [4.27943869e+01, 1.49257424e+01, 7.10816723e+00, ...,
        1.13239244e-01, 1.13239244e-01, 5.83545467e-02],
       [6.61412412e+01, 2.27700115e+01, 1.08069083e+01, ...,
        3.65238676e-01, 3.65238676e-01, 2.34784706e-01],
       ...,
       [7.09804143e+01, 2.56071809e+01, 1.05493425e+01, ...,
        1.01014810e-01, 1.01014810e-01, 1.07246400e-01],
       [1.12478540e+01, 3.37613436e+00, 2.67961234e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.20864676e-03],
       [3.75005766e+01, 1.31940683e+01, 8.47280930e+00, ...,
        3.98994887e-02, 3.98994887e-02, 9.16359183e-02]])

In [23]:
user_predicted_ratings.shape

(2071, 12911)

In [24]:
user_predicted_ratings

array([[9.55212958e+00, 3.29449494e+00, 1.47095436e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.45759814e-02],
       [4.27943869e+01, 1.49257424e+01, 7.10816723e+00, ...,
        1.13239244e-01, 1.13239244e-01, 5.83545467e-02],
       [6.61412412e+01, 2.27700115e+01, 1.08069083e+01, ...,
        3.65238676e-01, 3.65238676e-01, 2.34784706e-01],
       ...,
       [7.09804143e+01, 2.56071809e+01, 1.05493425e+01, ...,
        1.01014810e-01, 1.01014810e-01, 1.07246400e-01],
       [1.12478540e+01, 3.37613436e+00, 2.67961234e+00, ...,
        0.00000000e+00, 0.00000000e+00, 3.20864676e-03],
       [3.75005766e+01, 1.31940683e+01, 8.47280930e+00, ...,
        3.98994887e-02, 3.98994887e-02, 9.16359183e-02]])

Since we are interested only in the movies not rated by the user, we will ignore the movies rated by the user by making it zero. 

In [25]:
user_final_rating = np.multiply(user_predicted_ratings,dummy_train)
user_final_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,9.552130,3.294495,1.470954,0.040189,1.020561,5.201704,1.659968,0.054285,0.349317,5.031108,...,0.000000,0.003304,0.008125,0.000000,0.033861,0.000000,0.000000,0.000000,0.000000,0.074576
2,0.000000,14.925742,7.108167,0.908528,4.979097,14.034879,7.099211,0.785965,1.635084,19.880080,...,0.024640,0.061503,0.013683,0.000000,0.054489,0.000000,0.017600,0.113239,0.113239,0.058355
3,66.141241,22.770011,10.806908,1.206505,6.685770,34.141321,9.858038,0.669255,2.676321,32.180909,...,0.171026,0.100944,0.124693,0.080384,0.133438,0.317882,0.122161,0.365239,0.365239,0.234785
4,0.000000,10.226084,2.658416,0.299828,2.207704,12.222046,3.007363,0.123162,0.998604,12.896807,...,0.324958,0.172648,0.064566,0.009314,0.211999,0.144026,0.232113,0.346767,0.346767,0.200223
5,0.000000,19.867664,17.681491,2.762963,12.826641,32.774931,16.062810,1.801925,3.842096,35.779545,...,0.073628,0.000000,0.025125,0.025513,0.044435,0.156123,0.052592,0.060123,0.060123,0.082602


### Finding the top 5 recommendation for the *user*

In [36]:
# Take the user ID as input.
user_input = int(input("Enter your user name"))
print(user_input)

1


In [37]:
user_final_rating.head(7)

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,9.552130,3.294495,1.470954,0.040189,1.020561,5.201704,1.659968,0.054285,0.349317,5.031108,...,0.000000,0.003304,0.008125,0.000000,0.033861,0.000000,0.000000,0.000000,0.000000,0.074576
2,0.000000,14.925742,7.108167,0.908528,4.979097,14.034879,7.099211,0.785965,1.635084,19.880080,...,0.024640,0.061503,0.013683,0.000000,0.054489,0.000000,0.017600,0.113239,0.113239,0.058355
3,66.141241,22.770011,10.806908,1.206505,6.685770,34.141321,9.858038,0.669255,2.676321,32.180909,...,0.171026,0.100944,0.124693,0.080384,0.133438,0.317882,0.122161,0.365239,0.365239,0.234785
4,0.000000,10.226084,2.658416,0.299828,2.207704,12.222046,3.007363,0.123162,0.998604,12.896807,...,0.324958,0.172648,0.064566,0.009314,0.211999,0.144026,0.232113,0.346767,0.346767,0.200223
5,0.000000,19.867664,17.681491,2.762963,12.826641,32.774931,16.062810,1.801925,3.842096,35.779545,...,0.073628,0.000000,0.025125,0.025513,0.044435,0.156123,0.052592,0.060123,0.060123,0.082602
6,54.596352,18.590832,10.839627,1.625053,6.980606,19.860430,9.067216,0.607714,2.267193,22.297943,...,0.100707,0.004889,0.009237,0.082974,0.033177,0.139444,0.071933,0.122900,0.122900,0.091053
7,38.539643,21.516050,6.704593,2.044780,5.817238,16.263696,10.569066,2.010323,1.381648,0.000000,...,0.187936,0.187699,0.015714,0.031614,0.000000,0.007392,0.134240,0.000000,0.000000,0.042459


In [38]:
d = user_final_rating.loc[user_input].sort_values(ascending=False)[0:5]
d

movieId
296     21.108130
2571    19.782259
318     19.307398
2959    18.667860
593     18.643623
Name: 1, dtype: float64

In [39]:
#Mapping with Movie Title / Genres 
movie_mapping = pd.read_csv('https://raw.githubusercontent.com/antrikshsaxena/NLPCapstone/main/movies.csv')
movie_mapping.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
d = pd.merge(d,movie_mapping,left_on='movieId',right_on='movieId', how = 'left')
d.head()

,movieId,1,title,genres
0,296,21.108130,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,2571,19.782259,"Matrix, The (1999)",Action|Sci-Fi|Thriller
2,318,19.307398,"Shawshank Redemption, The (1994)",Crime|Drama
3,2959,18.667860,Fight Club (1999),Action|Crime|Drama|Thriller
4,593,18.643623,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller


# Evaluation - User User 

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [41]:
# Find out the common users of test and train dataset.
common = test[test.userId.isin(train.userId)]
common.shape

(90038, 4)

In [42]:
common.head()

,userId,movieId,rating,timestamp
29643,226,3156,1.0,1059516139
152649,1074,2194,3.0,906133915
123175,886,4886,3.5,1168350634
23712,185,1101,4.0,1191923488
99726,757,908,3.5,1184016903


In [43]:
# convert into the user-movie matrix.
common_user_based_matrix = common.pivot_table(index='userId', columns='movieId', values='rating')

In [44]:
common_user_based_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,204704,204780,205054,205106,205156,205383,205499,206499,206805,207309
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Convert the user_correlation matrix into dataframe.
user_correlation_df = pd.DataFrame(user_correlation)

In [46]:
user_correlation_df.shape

(2071, 2071)

In [47]:
user_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
0,1.000000,0.001899,0.016571,0.000000,0.000000,0.000000,0.039761,0.000000,0.003991,0.000000,...,0.0,0.000000,0.000829,0.000000,0.053598,0.015322,0.000000,0.020284,0.00000,0.004454
1,0.001899,1.000000,0.024391,0.014429,0.009537,0.034863,0.000000,0.000000,0.000000,0.003106,...,0.0,0.051286,0.036513,0.029803,0.000000,0.045524,0.010638,0.034615,0.00000,0.000000
2,0.016571,0.024391,1.000000,0.062999,0.040416,0.009671,0.000000,0.017017,0.000000,0.036063,...,0.0,0.061911,0.017470,0.083151,0.038958,0.060932,0.028651,0.112607,0.00000,0.017203
3,0.000000,0.014429,0.062999,1.000000,0.000000,0.011082,0.000000,0.000000,0.000000,0.027750,...,0.0,0.013295,0.000000,0.003963,0.000000,0.022046,0.020043,0.045028,0.00000,0.004144
4,0.000000,0.009537,0.040416,0.000000,1.000000,0.051212,0.045804,0.088584,0.079276,0.133326,...,0.0,0.033921,0.103921,0.007723,0.000000,0.081722,0.106743,0.014523,0.03769,0.009734


In [48]:
df_subtracted.head(1)

movieId,1,2,3,4,5,6,7,8,9,10,...,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# df_subtracted.head()

In [50]:
user_correlation_df['userId'] = df_subtracted.index

user_correlation_df.set_index('userId',inplace=True)
user_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.001899,0.016571,0.000000,0.000000,0.000000,0.039761,0.000000,0.003991,0.000000,...,0.0,0.000000,0.000829,0.000000,0.053598,0.015322,0.000000,0.020284,0.00000,0.004454
2,0.001899,1.000000,0.024391,0.014429,0.009537,0.034863,0.000000,0.000000,0.000000,0.003106,...,0.0,0.051286,0.036513,0.029803,0.000000,0.045524,0.010638,0.034615,0.00000,0.000000
3,0.016571,0.024391,1.000000,0.062999,0.040416,0.009671,0.000000,0.017017,0.000000,0.036063,...,0.0,0.061911,0.017470,0.083151,0.038958,0.060932,0.028651,0.112607,0.00000,0.017203
4,0.000000,0.014429,0.062999,1.000000,0.000000,0.011082,0.000000,0.000000,0.000000,0.027750,...,0.0,0.013295,0.000000,0.003963,0.000000,0.022046,0.020043,0.045028,0.00000,0.004144
5,0.000000,0.009537,0.040416,0.000000,1.000000,0.051212,0.045804,0.088584,0.079276,0.133326,...,0.0,0.033921,0.103921,0.007723,0.000000,0.081722,0.106743,0.014523,0.03769,0.009734


In [51]:
common.head(1)

,userId,movieId,rating,timestamp
29643,226,3156,1.0,1059516139


In [52]:
list_name = common.userId.tolist()

user_correlation_df.columns = df_subtracted.index.tolist()


user_correlation_df_1 =  user_correlation_df[user_correlation_df.index.isin(list_name)]

In [53]:
user_correlation_df_1.shape

(2071, 2071)

In [54]:
user_correlation_df_2 = user_correlation_df_1.T[user_correlation_df_1.T.index.isin(list_name)]

In [55]:
user_correlation_df_3 = user_correlation_df_2.T

In [56]:
user_correlation_df_3.head()

,1,2,3,4,5,6,7,8,9,10,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.001899,0.016571,0.000000,0.000000,0.000000,0.039761,0.000000,0.003991,0.000000,...,0.0,0.000000,0.000829,0.000000,0.053598,0.015322,0.000000,0.020284,0.00000,0.004454
2,0.001899,1.000000,0.024391,0.014429,0.009537,0.034863,0.000000,0.000000,0.000000,0.003106,...,0.0,0.051286,0.036513,0.029803,0.000000,0.045524,0.010638,0.034615,0.00000,0.000000
3,0.016571,0.024391,1.000000,0.062999,0.040416,0.009671,0.000000,0.017017,0.000000,0.036063,...,0.0,0.061911,0.017470,0.083151,0.038958,0.060932,0.028651,0.112607,0.00000,0.017203
4,0.000000,0.014429,0.062999,1.000000,0.000000,0.011082,0.000000,0.000000,0.000000,0.027750,...,0.0,0.013295,0.000000,0.003963,0.000000,0.022046,0.020043,0.045028,0.00000,0.004144
5,0.000000,0.009537,0.040416,0.000000,1.000000,0.051212,0.045804,0.088584,0.079276,0.133326,...,0.0,0.033921,0.103921,0.007723,0.000000,0.081722,0.106743,0.014523,0.03769,0.009734


In [57]:
user_correlation_df_3.shape

(2071, 2071)

In [58]:
user_correlation_df_3[user_correlation_df_3<0]=0

common_user_predicted_ratings = np.dot(user_correlation_df_3, common_user_based_matrix.fillna(0))
common_user_predicted_ratings

array([[5.10460224e+00, 1.74539764e+00, 6.96831596e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.02224987e+01, 6.74419414e+00, 2.26162307e+00, ...,
        1.99325288e-01, 0.00000000e+00, 0.00000000e+00],
       [3.57707385e+01, 1.15467646e+01, 2.97886475e+00, ...,
        4.02202154e-01, 1.42784707e-01, 1.42784707e-01],
       ...,
       [3.42388737e+01, 1.00466459e+01, 4.12806648e+00, ...,
        2.66442361e-01, 3.56811942e-02, 3.56811942e-02],
       [3.66282688e+00, 1.23265539e+00, 1.14883522e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.83078030e+01, 7.19569850e+00, 2.61970557e+00, ...,
        6.05070641e-02, 7.60002285e-02, 7.60002285e-02]])

In [59]:
dummy_test = common.copy()

dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x>=1 else 0)

dummy_test = dummy_test.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)

In [60]:
dummy_test.shape

(2071, 9529)

In [61]:
common_user_based_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,204704,204780,205054,205106,205156,205383,205499,206499,206805,207309
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
dummy_test.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,204704,204780,205054,205106,205156,205383,205499,206499,206805,207309
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
common_user_predicted_ratings = np.multiply(common_user_predicted_ratings,dummy_test)

In [59]:
common_user_predicted_ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,39,40,41,42,...,200614,200618,200768,200818,200938,201242,201436,201586,201588,201646,201749,201773,201811,202103,202263,202429,202439,202495,203048,203208,203222,203411,203519,203649,203751,204352,204580,204692,204698,204702,204704,204780,205054,205106,205156,205383,205499,206499,206805,207309
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35.770738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.20301,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,36.388331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Calculating the RMSE for only the movies rated by user. For RMSE, normalising the rating to (1,5) range.

In [60]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_user_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:375: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:376: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


MinMaxScaler(copy=True, feature_range=(1, 5))
[[     nan      nan      nan ...      nan      nan      nan]
 [     nan      nan      nan ...      nan      nan      nan]
 [3.563153      nan      nan ...      nan      nan      nan]
 ...
 [     nan      nan      nan ...      nan      nan      nan]
 [     nan      nan      nan ...      nan      nan      nan]
 [     nan      nan      nan ...      nan      nan      nan]]


In [61]:
common_ = common.pivot_table(index='userId', columns='movieId', values='rating')

In [62]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [63]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

1.4350750583833054


## Using Item similarity

# Item Based Similarity

Taking the transpose of the rating matrix to normalize the rating around the mean for different movie ID. In the user based similarity, we had taken mean for each user instead of each movie. 

In [ ]:
df_pivot = train.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).T

df_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,3.0,4.0,NaN,NaN,4.0,NaN,3.5,NaN,4.0,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalising the movie rating for each movie for using the Adujsted Cosine

In [ ]:
mean = np.nanmean(df_pivot, axis=1)
df_subtracted = (df_pivot.T-mean).T

In [ ]:
df_subtracted.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,-0.45501,NaN,-0.95501,0.04499,NaN,NaN,0.044990,NaN,-0.45501,NaN,0.044990,0.04499,NaN,NaN,NaN,NaN,-0.95501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.95501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.04499,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.95501,NaN,NaN,NaN,0.04499,-0.95501,NaN,NaN,NaN,NaN,NaN,NaN,-0.95501,NaN,-0.45501,NaN,NaN,NaN,NaN,NaN,-0.455010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.04499,NaN,NaN,0.044990,NaN,NaN,NaN,-0.95501
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.610811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.110811,-0.889189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.389189,-0.389189,NaN,NaN,NaN,-0.889189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.889189,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.816667,NaN,NaN,NaN,-1.183333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.183333,NaN,-1.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.183333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.816667,NaN,NaN,NaN,NaN,-0.183333,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.133663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.133663,NaN,NaN,NaN,-0.133663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Finding the cosine similarity using pairwise distances approach

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(df_subtracted.fillna(0), metric='cosine')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation)

[[ 1.          0.10291474 -0.00527634 ...  0.          0.
   0.        ]
 [ 0.10291474  1.          0.03170014 ...  0.          0.
   0.        ]
 [-0.00527634  0.03170014  1.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [ ]:
item_correlation.shape

(12911, 12911)

Filtering the correlation only for which the value is greater than 0. (Positively correlated)

In [ ]:
item_correlation[item_correlation<0]=0
item_correlation

array([[1.        , 0.10291474, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10291474, 1.        , 0.03170014, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03170014, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# Prediction - Item Item

In [ ]:
item_predicted_ratings = np.dot((df_pivot.fillna(0).T),item_correlation)
item_predicted_ratings

array([[ 5.83679879,  2.41464654,  1.94850435, ...,  0.        ,
         0.        ,  0.        ],
       [27.32838057, 18.73037026,  6.83694139, ...,  0.        ,
         0.        ,  0.        ],
       [71.07549452, 50.83081163, 16.86416369, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [21.92022103, 20.39922913,  7.56019237, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.34159079,  0.84726105,  0.66059729, ...,  0.        ,
         0.        ,  0.        ],
       [16.64338891, 12.69091959,  5.81227628, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
item_predicted_ratings.shape

(2071, 12911)

In [ ]:
dummy_train.shape

(2071, 12911)

### Filtering the rating only for the movies not rated by the user for recommendation

In [ ]:
item_final_rating = np.multiply(item_predicted_ratings,dummy_train)
item_final_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,38,39,40,41,...,202439,202599,202759,202934,203066,203182,203204,203212,203218,203222,203244,203322,203375,203513,203519,203723,203950,204198,204294,204342,204504,204542,204698,204832,205072,205383,205413,205557,205583,205945,205967,206272,206293,206499,206523,206845,206861,207309,208002,208793
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.836799,2.414647,1.948504,1.053313,1.664116,2.900967,1.464508,0.875935,1.481628,3.313284,1.102115,1.359496,1.377499,1.946804,1.336029,5.331667,3.172505,2.304680,2.634617,1.016137,2.409931,1.691307,1.918475,1.416901,1.893220,1.320966,1.183438,2.636449,3.362342,0.822741,2.399006,4.394929,0.0,3.725530,1.615948,3.287521,0.000000,3.395454,3.145430,2.419277,...,1.179661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.546872,0.0,0.701731,0.0,0.081968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.439799,0.0,0.0,0.457219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,18.730370,6.836941,9.363118,8.222216,13.292342,9.611464,3.705273,9.897551,21.484258,12.570877,10.287221,7.866425,9.498181,5.334188,13.193283,9.741122,4.812225,14.879701,12.377068,12.409858,10.931229,9.433581,11.600822,9.048906,4.787380,4.438859,6.193937,6.149578,5.683170,7.751338,14.194265,0.0,21.333264,4.433457,10.948242,2.440009,16.064186,11.429770,7.405310,...,4.268891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125503,0.0,3.524765,0.0,1.660968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.640216,0.0,0.0,9.247180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,71.075495,50.830812,16.864164,36.129112,17.996283,36.056161,19.455536,26.062272,32.047686,42.683568,27.520109,61.090367,39.547316,31.233267,5.282064,45.232143,37.386436,16.083796,40.785107,16.185328,26.430657,42.244606,53.017020,26.684270,19.123080,10.848638,16.203299,19.879640,19.295139,14.518257,28.043347,0.000000,0.0,47.039861,5.779081,33.667706,1.008682,35.253629,44.934213,28.206781,...,27.954654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.657680,0.0,53.357426,0.0,5.268821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.277304,0.0,0.0,48.294710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,16.289441,6.163784,6.882548,5.744073,13.938653,5.300006,8.632429,8.484791,15.868870,7.068904,18.022130,10.858236,10.340895,2.142233,17.976587,12.023357,3.266670,15.553435,7.725462,12.171769,9.115309,17.054987,7.449948,6.094116,3.761508,4.607504,7.811523,5.119653,5.381298,7.681701,18.512415,0.0,17.331468,1.890455,11.455449,1.052716,14.858025,12.627695,7.829542,...,8.776493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.433791,0.0,19.421290,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.619777,0.0,0.0,17.844855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,14.295191,7.947143,6.407635,7.038309,7.613087,5.866075,5.893830,8.269718,14.117642,12.441575,7.698015,5.541342,4.908702,4.499263,10.298705,7.104078,8.378062,0.000000,9.927177,9.891820,11.969391,9.993500,10.590708,8.241523,3.209154,5.390730,2.856468,5.802585,5.957798,11.551173,8.406005,0.0,13.986785,4.263240,0.000000,3.780995,0.000000,10.168428,8.421631,...,1.497611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.231581,0.0,1.206331,0.0,0.702955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.815940,0.0,0.0,4.258672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Finding the top 5 recommendation for the *user*



In [ ]:
# Take the user ID as input
user_input = int(input("Enter your user name"))
print(user_input)

Enter your user name1
1


In [ ]:
# Recommending the Top 5 products to the user.
d = item_final_rating.loc[user_input].sort_values(ascending=False)[0:5]
d

movieId
27834    11.945502
7637     11.690961
3559     11.530296
5269     11.355734
6440     11.349935
Name: 1, dtype: float64

In [ ]:
#Mapping with Movie Title / Genres 
movie_mapping = pd.read_csv('https://raw.githubusercontent.com/antrikshsaxena/NLPCapstone/main/movies.csv', encoding='latin-1')

In [ ]:
d = pd.merge(d,movie_mapping,left_on='movieId',right_on='movieId',how = 'left')
d.head()

,movieId,1,title,genres
0,27834,11.945502,"Return, The (Vozvrashcheniye) (2003)",Drama
1,7637,11.690961,Irma Vep (1996),Comedy|Drama
2,3559,11.530296,Limelight (1952),Comedy|Drama|Romance
3,5269,11.355734,"Piano Teacher, The (La pianiste) (2001)",Drama
4,6440,11.349935,Barton Fink (1991),Drama|Thriller


In [ ]:
train_new = pd.merge(train,movie_mapping,left_on='movieId',right_on='movieId',how='left')
train_new[train_new.userId == 1] .head()

,userId,movieId,rating,timestamp,title,genres
2866,1,8973,4.0,1147869211,Bad Education (La mala educaciÃ³n) (2004),Drama|Thriller
4548,1,7318,2.0,1147879850,"Passion of the Christ, The (2004)",Drama
11584,1,4144,5.0,1147868898,In the Mood For Love (Fa yeung nin wa) (2000),Drama|Romance
13280,1,8327,5.0,1147879375,Dolls (2002),Drama|Romance
17280,1,1260,3.5,1147877857,M (1931),Crime|Film-Noir|Thriller


# Evaluation - Item Item

Evaluation will we same as you have seen above for the prediction. The only difference being, you will evaluate for the movie already rated by the user insead of predicting it for the movie not rated by the user. 

In [ ]:
test.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [ ]:
common =  test[test.movieId.isin(train.movieId)]
common.shape

(88035, 4)

In [ ]:
common.head(4)

,userId,movieId,rating,timestamp
29643,226,3156,1.0,1059516139
152649,1074,2194,3.0,906133915
123175,886,4886,3.5,1168350634
23712,185,1101,4.0,1191923488


In [ ]:
common_item_based_matrix = common.pivot_table(index='userId', columns='movieId', values='rating').T

In [ ]:
common_item_based_matrix.shape

(7783, 2071)

In [ ]:
item_correlation_df = pd.DataFrame(item_correlation)

In [ ]:
item_correlation_df.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,12871,12872,12873,12874,12875,12876,12877,12878,12879,12880,12881,12882,12883,12884,12885,12886,12887,12888,12889,12890,12891,12892,12893,12894,12895,12896,12897,12898,12899,12900,12901,12902,12903,12904,12905,12906,12907,12908,12909,12910
0,1.0,0.102915,0.0,0.00434,0.095113,0.066226,0.0,0.009237,0.02017,0.110567,0.001906,0.01273,0.037987,0.0,0.021544,0.018412,0.023088,0.0,0.083921,0.0,0.036459,0.011627,0.111087,0.004441,0.024696,0.022684,0.019995,0.0,0.0,0.0,0.045761,0.06942,0.0,0.109849,0.0,0.08708,0.001596,0.046534,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
item_correlation_df['movieId'] = df_subtracted.index
item_correlation_df.set_index('movieId',inplace=True)
item_correlation_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,12871,12872,12873,12874,12875,12876,12877,12878,12879,12880,12881,12882,12883,12884,12885,12886,12887,12888,12889,12890,12891,12892,12893,12894,12895,12896,12897,12898,12899,12900,12901,12902,12903,12904,12905,12906,12907,12908,12909,12910
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.102915,0.000000,0.004340,0.095113,0.066226,0.000000,0.009237,0.020170,0.110567,0.001906,0.012730,0.037987,0.00000,0.021544,0.018412,0.023088,0.000000,0.083921,0.000000,0.036459,0.011627,0.111087,0.004441,0.024696,0.022684,0.019995,0.000000,0.000000,0.0,0.045761,0.069420,0.0,0.109849,0.0,0.08708,0.001596,0.046534,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001596,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003606,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.102915,1.000000,0.031700,0.000000,0.060859,0.046665,0.030885,0.025288,0.022028,0.157213,0.075537,0.073021,0.011394,0.02182,0.090197,0.057968,0.022819,0.039248,0.075351,0.050777,0.075855,0.149411,0.122759,0.125354,0.002899,0.037272,0.046453,0.000987,0.000000,0.0,0.068773,0.012696,0.0,0.121856,0.0,0.01059,0.000000,0.078514,0.000000,0.000000,...,0.020464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.10963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.031700,1.000000,0.004382,0.069432,0.011788,0.054688,0.060193,0.100534,0.011718,0.061175,0.000000,0.000000,0.00000,0.000000,0.050120,0.002268,0.020197,0.106894,0.056241,0.002759,0.021814,0.063946,0.073520,0.000000,0.024446,0.128107,0.009599,0.034253,0.0,0.057190,0.000000,0.0,0.033450,0.0,0.00000,0.056231,0.000000,0.035462,0.032828,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.012623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.004340,0.000000,0.004382,1.000000,0.038519,0.029985,0.019847,0.000000,0.067890,0.000000,0.000000,0.019552,0.000000,0.00000,0.094436,0.000000,0.007044,0.052955,0.000000,0.000000,0.000000,0.081803,0.009588,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.047059,0.000385,0.0,0.030021,0.0,0.00000,0.000000,0.040589,0.000000,0.013528,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.095113,0.060859,0.069432,0.038519,1.000000,0.063813,0.000000,0.000000,0.043664,0.091824,0.070592,0.000000,0.000000,0.00000,0.040252,0.053018,0.000000,0.000000,0.041148,0.000000,0.041346,0.000000,0.007879,0.000000,0.000000,0.069025,0.077936,0.009779,0.023431,0.0,0.065998,0.000000,0.0,0.039640,0.0,0.00000,0.063713,0.020557,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
list_name = common.movieId.tolist()

In [ ]:
item_correlation_df.columns = df_subtracted.index.tolist()

item_correlation_df_1 =  item_correlation_df[item_correlation_df.index.isin(list_name)]

In [ ]:
item_correlation_df_2 = item_correlation_df_1.T[item_correlation_df_1.T.index.isin(list_name)]

item_correlation_df_3 = item_correlation_df_2.T

In [ ]:
item_correlation_df_3.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,40,41,42,43,...,193950,193958,194004,194016,194238,194448,194951,194959,195159,195163,195295,195777,196889,196891,196997,197199,197201,197491,197537,197691,197709,197711,197879,198703,200306,200540,200818,201588,201646,201749,201773,201811,202429,202439,203222,203519,204698,205383,206499,207309
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.102915,0.000000,0.004340,0.095113,0.066226,0.000000,0.009237,0.020170,0.110567,0.001906,0.012730,0.037987,0.00000,0.021544,0.018412,0.023088,0.000000,0.083921,0.000000,0.036459,0.011627,0.111087,0.004441,0.024696,0.022684,0.019995,0.000000,0.000000,0.0,0.045761,0.069420,0.109849,0.0,0.08708,0.046534,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.023605,0.101598,0.0,0.000000,0.000000,0.0,0.012044,0.0,0.0,0.069336,0.001671,0.017733,0.01172,0.016137,0.070932,0.0,0.0,0.0,0.0,0.018821,0.0,0.0,0.000000,0.0000,0.0,0.049834,0.015703,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.003606,0.00000,0.0,0.0
2,0.102915,1.000000,0.031700,0.000000,0.060859,0.046665,0.030885,0.025288,0.022028,0.157213,0.075537,0.073021,0.011394,0.02182,0.090197,0.057968,0.022819,0.039248,0.075351,0.050777,0.075855,0.149411,0.122759,0.125354,0.002899,0.037272,0.046453,0.000987,0.000000,0.0,0.068773,0.012696,0.121856,0.0,0.01059,0.078514,0.000000,0.000000,0.000000,0.000459,...,0.0,0.0,0.000000,0.000000,0.0,0.003918,0.000000,0.0,0.005254,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0516,0.0,0.028379,0.027660,0.0,0.0,0.035445,0.05432,0.020464,0.0,0.0,0.000000,0.10963,0.0,0.0
3,0.000000,0.031700,1.000000,0.004382,0.069432,0.011788,0.054688,0.060193,0.100534,0.011718,0.061175,0.000000,0.000000,0.00000,0.000000,0.050120,0.002268,0.020197,0.106894,0.056241,0.002759,0.021814,0.063946,0.073520,0.000000,0.024446,0.128107,0.009599,0.034253,0.0,0.057190,0.000000,0.033450,0.0,0.00000,0.000000,0.035462,0.032828,0.051138,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.034916,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.034427,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
4,0.004340,0.000000,0.004382,1.000000,0.038519,0.029985,0.019847,0.000000,0.067890,0.000000,0.000000,0.019552,0.000000,0.00000,0.094436,0.000000,0.007044,0.052955,0.000000,0.000000,0.000000,0.081803,0.009588,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.047059,0.000385,0.030021,0.0,0.00000,0.040589,0.000000,0.013528,0.000000,0.003085,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.389882,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
5,0.095113,0.060859,0.069432,0.038519,1.000000,0.063813,0.000000,0.000000,0.043664,0.091824,0.070592,0.000000,0.000000,0.00000,0.040252,0.053018,0.000000,0.000000,0.041148,0.000000,0.041346,0.000000,0.007879,0.000000,0.000000,0.069025,0.077936,0.009779,0.023431,0.0,0.065998,0.000000,0.039640,0.0,0.00000,0.020557,0.000000,0.000000,0.029413,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.003768,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [ ]:
item_correlation_df_3[item_correlation_df_3<0]=0
common_item_predicted_ratings = np.dot(item_correlation_df_3, common_item_based_matrix.fillna(0))
common_item_predicted_ratings

array([[ 1.10153262, 12.83252349, 35.8647666 , ...,  9.1484491 ,
         0.51093827,  9.15792271],
       [ 0.39077366,  9.74879386, 22.62663044, ...,  7.05031014,
         0.92552995,  6.28294116],
       [ 0.11700293,  4.30626816,  8.2908921 , ...,  3.77831808,
         0.26364204,  3.77408128],
       ...,
       [ 0.43354838,  3.97316391, 22.48683777, ...,  6.770097  ,
         0.1689317 ,  4.1952958 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
common_item_predicted_ratings.shape

(7783, 2071)

Dummy test will be used for evaluation. To evaluate, we will only make prediction on the movies rated by the user. So, this is marked as 1. This is just opposite of dummy_train



In [ ]:
dummy_test = common.copy()

dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x>=1 else 0)

dummy_test = dummy_test.pivot_table(index='userId', columns='movieId', values='rating').T.fillna(0)

common_item_predicted_ratings = np.multiply(common_item_predicted_ratings,dummy_test)

The products not rated is marked as 0 for evaluation. And make the item- item matrix representaion.


In [ ]:
common_ = common.pivot_table(index='userId', columns='movieId', values='rating').T

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from numpy import *

X  = common_item_predicted_ratings.copy() 
X = X[X>0]

scaler = MinMaxScaler(feature_range=(1, 5))
print(scaler.fit(X))
y = (scaler.transform(X))

print(y)

MinMaxScaler(copy=True, feature_range=(1, 5))
[[       nan        nan 2.52545034 ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 ...
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]
 [       nan        nan        nan ...        nan        nan        nan]]


In [ ]:
# Finding total non-NaN value
total_non_nan = np.count_nonzero(~np.isnan(y))

In [ ]:
rmse = (sum(sum((common_ - y )**2))/total_non_nan)**0.5
print(rmse)

1.3169197240439345


##**Summary - Recommendation Engine**